## Code

In [25]:
## 1. Import Libraries
import time
from configparser import ConfigParser
from fyers_api import fyersModel
from fyers_api.Websocket import ws
import sys
import sqlite3
import pandas as pd
import os
import talib              
import datetime as dt
import pdb
import asyncio
print('1. Libraries Imported')

## 2. Load Access Token
config = ConfigParser()
config.read('config.ini')

with open("access.txt", 'r') as r:
    access_token = r.read()

print("Access_token:", access_token)
print('2. Access Token Imported')

client_id='L0QBIDUTN6-100'
fyers = fyersModel.FyersModel(token=access_token, log_path=os.getcwd(), client_id = client_id)
validation = fyers.get_profile()

## 3. validation
print("Validation:", validation)
print("3. Connection Validated")


## 4. Establishing connection with sqlite
db = sqlite3.connect('Stratergy.db', check_same_thread=False)
c=db.cursor()
print("4. Connection Established with Sqlite")


## 5. Creating Table in sqlite
s = ['RSI']

def create_tables(tokens):
    for i in tokens:
        print(i)
        # Drop the table if it exists, then create the new table
        c.execute("DROP TABLE IF EXISTS {};".format(i))
        c.execute(f"CREATE TABLE {i} (date DATETIME PRIMARY KEY, open REAL(15,5), high REAL(15,5), low REAL(15,5), close REAL(15,5), volume INTEGER, rsi REAL(15,5), '-2candle' TEXT, '-1candle' TEXT, 'gap' TEXT,local_time DATETIME, 'banknifty_symbol' TEXT, 'rsi_period' int, 'call_rsi_threshold' int, 'put_rsi_threshold' int, 'Contract' TEXT, 'call_symbol' TEXT, 'put_symbol' TEXT);")
    try:
        db.commit()
    except:
        db.rollback()

create_tables(s)
print("5. Table created:", s)



## 6. RSI Stratergy variables
banknifty_symbol = "NSE:NIFTYBANK-INDEX"
is_long_trade = False
is_short_trade = False
rsi_period = 14
call_rsi_threshold = 60
put_rsi_threshold = 58

print("6. RSI Startergy Configured as symbol:",banknifty_symbol,"| rsi_period:",rsi_period, "| upper_limit:",call_rsi_threshold,"| lower_limit:",put_rsi_threshold)


## 7. Symbol data for historical candles
symbol_data = {
    "symbol": banknifty_symbol,
    "resolution": "1",
    "date_format": "1",
    "range_from": "2023-11-23",
    "range_to": "2023-11-23",
    "cont_flag": "1"
}
print("7. Symbol data for historical candles are configured")


## 8. Data Creation in DF
def process_data(data):
    global is_long_trade, is_short_trade, call_symbol, put_symbol

    df = pd.DataFrame(data['candles'])
    print("DataFrame:")
    df[0] = pd.to_datetime(df[0],unit='s')
    df[0]=(df[0].dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata'))
    df[0] = df[0].dt.tz_localize(None)
    df.columns=['date','open','high','low','close','volume']
    
    df['rsi'] = talib.RSI(df['close'], timeperiod=rsi_period)
    df['-2candle'] = df['rsi'].iloc[-2]  
    df['-1candle'] = df['rsi'].iloc[-1] 
    

    # df['local_time'] = time.strftime('%Y-%m-%d %H:%M:%S')
    df= df.set_index('date')
    
    print("8. Data Created in df")
    print(df)
    
    print(str(time.localtime().tm_hour) + ':' + str(time.localtime().tm_min) + ':' + str(time.localtime().tm_sec) +
          " RSI (-2candle) :" + str(df['rsi'].iloc[-2]) + " RSI (-1 candle) : " + str(df['rsi'].iloc[-1]) + " RSI (candle) : " + str(df['rsi'].iloc[0]))   
    
 ## 8.1. Call Entry
    if not is_long_trade and df['rsi'].iloc[-2] < call_rsi_threshold and df['rsi'].iloc[-1] > call_rsi_threshold:
        ## for long -1> RSI >-2    eg: -2(58) | RSI(60) | -1(62)   
        # Place long trade
        print("RSI crossover: Buy Bank Nifty / Buy Call Option")
        round_to_strike = int(round(float(df['close'].iloc[-1]), -2))
        call_symbol = "NSE:BANKNIFTY23N31" + str(round_to_strike) + 'CE'
        trade = {
            "symbol": call_symbol,
            "qty": 1,
            "type": 2,
            "side": 1,
            "productType": "INTRADAY",
            "limitPrice": 0,
            "stopPrice": 0,
            "validity": "DAY",
            "disclosedQty": 0,
            "offlineOrder": "False",
            "stopLoss": 0,
            "takeProfit": 0
        }
        order = fyers.place_order(data=trade)

        # Print relevant information for debugging
        print('Order parameters:', trade)
        print('Response:', order)

        # Set trade flags
        is_long_trade = True
        is_short_trade = False
        
    if is_long_trade:
        print("8.1 Long entry has been taken. Check conditions for Stop Loss or Target")
        # Write your logic for stop loss
        # is_long_trade = False    
 
 
 ## 8.2. Short Entry   
    if not is_short_trade and df['rsi'].iloc[-2] > put_rsi_threshold and df['rsi'].iloc[-1] < put_rsi_threshold:
        ## for short -1< RSI <-2    eg: -2(62) | RSI(60) | -1(58) 
        # Place short trade
        print("RSI crossover: Sell Bank Nifty / Buy Put Option")
        round_to_strike = int(round(float(df['close'].iloc[-1]), -2))
        put_symbol = "NSE:BANKNIFTY23N31" + str(round_to_strike) + 'PE'
        trade = {
            "symbol": put_symbol,
            "qty": 1,
            "type": 2,
            "side": 1,
            "productType": "INTRADAY",
            "limitPrice": 0,
            "stopPrice": 0,
            "validity": "DAY",
            "disclosedQty": 0,
            "offlineOrder": "False",
            "stopLoss": 0,
            "takeProfit": 0
        }

        order = fyers.place_order(data=trade)

        # Print relevant information for debugging
        print('Order parameters:', trade)
        print('Response:', order)

        # Set trade flags
        is_long_trade = False
        is_short_trade = True
        

    if is_short_trade:
        print("8.2 Short entry has been taken. Check conditions for Stop Loss or Target")
        # Write your logic for stop loss
        # is_short_trade = False
        
    
##  9. Insering Raw Data into Table
    print("11. Data Insertion Started")

    for index, row in df.iterrows():
        try:
            
            # ltt = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(df['timestamp']))
            
            data = (index.strftime('%Y-%m-%d %H:%M:%S'), row['open'], row['high'], row['low'], row['close'],  row['volume'],
                    row['rsi'], row['-2candle'], row['-1candle'],'   ',time.strftime('%Y-%m-%d %H:%M:%S'),
                    banknifty_symbol, rsi_period,call_rsi_threshold,  put_rsi_threshold, 'NSE:BANKNIFTY23N22','   ','   ')

            print(f"INSERT INTO {s[0]} VALUES {data};")
            c.execute(f"INSERT INTO {s[0]} VALUES {data};")
            c.execute('UPDATE {} SET put_symbol = 1 WHERE "-2candle" > put_rsi_threshold AND "-1candle" < put_rsi_threshold'.format(s[0]))
            c.execute('UPDATE {} SET call_symbol = 1 WHERE "-2candle" < call_rsi_threshold AND "-1candle" > call_rsi_threshold'.format(s[0]))
            print("Data Inserted/Updated in",s)
        except Exception as e:
            print(e)
            pass

    try:
        db.commit()
    except:
        db.rollback()
         

## 8n call function
while True:
    data = fyers.history(data=symbol_data)
    process_data(data)
    time.sleep(10)  # Sleep for 10 seconds before the next iteration
    
    

# Notes-  (at start of 2nd minute, 1st min data will be avilable/dumped)
# long & short in database
# read order for stoploss


1. Libraries Imported
Access_token: eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE3MDE1MTcwMTIsImV4cCI6MTcwMTU2MzQ1MiwibmJmIjoxNzAxNTE3MDEyLCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbGF4YlV5YjFiNmg3U1RHUzBqRy1MTnFDektPYWQ3LU0wWElsQzJGcVZzd3hlbElyWENjeF9OY0JZRmt5dEZIYm5scklsVjBRQ21ZNVdmRVlTX1IxYmZXZXpkekZYS1g1WTZhX2xTQ3lmWnZMMGNVbz0iLCJkaXNwbGF5X25hbWUiOiJUVVNIQVIgVU1FU0ggU0VUSEkiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJhOGQ5ZTJjYzc1NWJhMmY1YjFiOThlYTkxNDdlY2MyMmZkMjI2ODVkMmM5NWY4YTA0MTk3ODVmYyIsImZ5X2lkIjoiWFQwOTk2MCIsImFwcFR5cGUiOjEwMCwicG9hX2ZsYWciOiJOIn0.XBJ6clQzx3ry0h19cPBgAMRcCiFwlWFpWDbe4kfyPqk
2. Access Token Imported
Validation: {'s': 'ok', 'code': 200, 'message': '', 'data': {'fy_id': 'XT09960', 'name': 'TUSHAR UMESH SETHI', 'image': None, 'display_name': None, 'pin_change_date': '31-10-2023 20:01:26', 'email_id': 'tusharsethi385@gmail.com', 'pwd_change_date': None, 'PAN': '-------

KeyboardInterrupt: 